<a href="https://colab.research.google.com/github/magus96/BCDProj/blob/master/BCDVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import pickle
import os.path
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import model_from_json
from sklearn import metrics
from sklearn.metrics import roc_curve,auc,accuracy_score,classification_report
from keras.optimizers import SGD
from keras import models
from sklearn import svm

In [2]:
!git clone https://github.com/magus96/BCDProj.git

fatal: destination path 'BCDProj' already exists and is not an empty directory.


In [3]:
cd '/content/BCDProj/'

/content/BCDProj


In [0]:
from helpers import resize_to_fit
path='/content/BCDProj/Test/'
data=[]
labels=[]
for image_file in paths.list_images(path):
    image=cv2.imread(image_file)
    image=resize_to_fit(image,224,224)
    label=image_file.split(os.path.sep)[-2]
    data.append(image)
    labels.append(label)

In [5]:
data = np.array(data, dtype="float") / 255.0
labels=np.array(labels)
print(data.shape)
print(labels.shape)
print(labels)

(1844, 224, 224, 3)
(1844,)
['1' '1' '1' ... '0' '0' '0']


In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [7]:
vgg_conv = VGG16(weights='imagenet',include_top=False,input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
nTrain=1844

In [0]:
train_features = np.zeros(shape=(nTrain, 7, 7, 512))
datagen.fit(data)


In [10]:
train_features=vgg_conv.predict(data)
print(train_features.shape)

(1844, 7, 7, 512)


In [0]:
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

In [12]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(train_features,labels)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [27]:
from helpers import resize_to_fit
path='/content/BCDProj/Validation/'
data1=[]
labels1=[]
for image_file in paths.list_images(path):
    image=cv2.imread(image_file)
    image=resize_to_fit(image,224,224)
    label=image_file.split(os.path.sep)[-2]
    data1.append(image)
    labels1.append(label)
    
data1 = np.array(data1, dtype="float") / 255.0
labels1 = np.array(labels1)
print(data1.shape)

(221, 224, 224, 3)


In [0]:
test_features=np.zeros(shape=(221,7,7,512))

In [29]:
test_features=vgg_conv.predict(data1)
print(test_features.shape)

(221, 7, 7, 512)


In [0]:
test_features = np.reshape(test_features, (221, 7 * 7 * 512))

In [31]:
preds=clf.predict(test_features)
print(preds)

['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '0' '1' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '0'
 '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0'
 '0' '1' '0' '0' '0' '0' '0' '0' '1' '1' '1' '0' '0' '0' '1' '0' '1' '0'
 '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '1']


In [32]:
print(accuracy_score(preds,labels1))

0.8959276018099548


In [0]:
filename = 'log_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [24]:
print(classification_report(labels1,preds))

              precision    recall  f1-score   support

           0       0.95      0.80      0.87        94
           1       0.87      0.97      0.91       127

   micro avg       0.90      0.90      0.90       221
   macro avg       0.91      0.88      0.89       221
weighted avg       0.90      0.90      0.89       221



TypeError: ignored